In [1]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import glob
from datetime import date, datetime, time, timedelta
from collections import defaultdict
import os

# reduccion de datos: sin mediciones durante la noche

existen algunas estaciones que pierden datos. Esto se vuelve un problema grave si estos datos perdidos son en el periodo de noche. Para nuestro trabajo, eliminaremos estos dias de la medicion dado que no nos sirven para entrenar.

* Consideraremos el rango de 20 hrs a 10 hrs (14 horas de intervalo)
* esperaremos tenemos almenos 8 hrs de medicion, sin importan si estan entre cortadas, esto significa, 32 datos
* para trabajar mas facilmente, desplazaremos el dia en +4 hrs, de forma que las 24 hrs sean las 00 horas del dia siguiente y las 10 hrs del dia siguiente sean las 14 hrs, esto facilitara el filtrado


In [2]:
# primero cargamos los datos limpiados en la iteracion anterior

path = "../data/cleared"
dfs = []
dict_meta = {"region": [], "estacion": [], "codigo": []}
for file in glob.glob(path + "/*/*"):
    print(file + " " * 30, end="\r")
    file = file.replace("\\", "/")
    dict_meta["region"].append(file.split("/")[-2])
    dict_meta["estacion"].append(file.split("/")[-1].split(".")[0])
    df = pd.read_csv(file)
    dict_meta["codigo"].append(df["Codigo"][0])
    dfs.append(df)

In [3]:
# luego juntaremos estos datos en un solo dataframe, ordenando por el par (codigo, fecha, hora)

df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df = df.reset_index(drop=True)

In [4]:
df.columns

Index(['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
       'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
       'Radiacion Solar', 'Presion Atmosferica'],
      dtype='object')

In [5]:
# obtenemos la matriz de los datos y agregamos 2 columnas extras, para el datetime y el datetime desplazado

data = df.values
dummy = np.full((data.shape[0], 2), None)
data = np.append(data, dummy, axis=1)

In [6]:
data.shape

(5301924, 12)

In [15]:
# entonces primero calculamos los datetime y los agregamos a la matriz de datos
# para gilizar, definimos multiprocessing

def worker(data, ini, end):
    print("starting worker range [{}, {}]... ".format(ini, end), end="")
#     ddtt = []
#     displaced_date = []
#     deltatime = timedelta(hours = 5)
#     c = 0
#     for i in range(ini, end):
#         d = data[i][1]
#         t = data[i][2]
#         if " " in d:
#             d = d.split(" ")[0]
#             datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#         elif "-" in d:
#             datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#         else:
#             datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
        
#         data[i][10] = (datm + deltatime).__str__()
#         data[i][11] = datm
# #         displaced_date.append((datm + deltatime).date().__str__())
# #         ddtt.append(datm)
#         c += 1
# #     out_q.put(c)
    print("done")
    
def run_compute_datetime(data, N):
    
    # obtenemos los indices de los subrangos
    print("obteniendo sub rangos ... ")
    r = data.shape[0] // N
    idxs = np.arange(N + 1) * r
    
    # lanzamos los workers
    print("starting workers ....")
    jobs = []
    for i in range(len(idxs)-1):
        ini = idxs[i]
        end = idxs[i+1]
        print("launching worker for [{},{}]".format(ini, end))
        jobs.append(mp.Process(target=worker, args=(data, ini, end)))
        jobs[-1].start()
    print("waiting workers ...")
    for p in jobs:
        p.join()
        

# ddtt = []
# displaced_date = []
# # create datetime object and displaceit by -19 hours
# deltatime = timedelta(hours = 5)
# c = 0
# print(len(ddate))
# for d, t in zip(ddate, dtime):
    
#     if c % 10000 == 0:
#         print(c, end="\r")
#     if " " in d:
#         d = d.split(" ")[0]
#         datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#     elif "-" in d:
#         datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#     else:
#         datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
#     displaced_date.append((datm + deltatime).date().__str__())
#     ddtt.append(datm)
#     c += 1
# compute_datetime(data, 0, 50, None)
# data
run_compute_datetime(data, 24)  
# df["datetime"] = ddtt
# df["displaced_date"] = displaced_date
# df.count()

obteniendo sub rangos ... 
starting workers ....
launching worker for [0,220913]


BrokenPipeError: [Errno 32] Broken pipe

In [34]:
data

array([[13, '2010-06-11', '12:15:00', ..., 1008.0, '2010-06-11 17:15:00',
        datetime.datetime(2010, 6, 11, 12, 15)],
       [13, '2010-06-11', '12:30:00', ..., 1007.0, '2010-06-11 17:30:00',
        datetime.datetime(2010, 6, 11, 12, 30)],
       [13, '2010-06-11', '12:45:00', ..., 1007.0, '2010-06-11 17:45:00',
        datetime.datetime(2010, 6, 11, 12, 45)],
       ...,
       [450050, '2010-06-30', '02:30:00', ..., 1016.0, None, None],
       [450050, '2010-06-30', '02:45:00', ..., 1015.9, None, None],
       [450050, '2010-06-30', '03:00:00', ..., 1016.0, None, None]],
      dtype=object)

In [ ]:
datetime_arr = []
displaced_date_arr = []
qlen = rqueue.qsize()
while qlen > 0:
    ddtt_v, dispddtt_v = rqueue.get()
    datetime_arr.extend(ddtt_v)
    displaced_date_arr.extend(dispddtt_v)
    


In [16]:
a = 100
N = 20 
r = a // N
x = np.arange(N) * r
x

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
       85, 90, 95])

In [7]:
df2v = df.drop_duplicates(subset=['Codigo', 'Fecha', 'Hora'], keep="first")
df2v.count()

Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64